In [25]:
from astroquery.simbad import Simbad
from astropy import units as u 
from astropy.coordinates import SkyCoord


customsimbad=Simbad()
customsimbad.add_votable_fields('plx')

stars=["Alpha Cen A", "Alpha Cen B", "Proxima Cen"]


result=customsimbad.query_objects(stars)

result




C:\Users\supar\AppData\Local\Temp\ipykernel_10032\3733245359.py:7: DeprecationWarning: 'plx' has been renamed 'plx_value'. You'll see it appearing with its new name in the output table
  customsimbad.add_votable_fields('plx')


main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode,plx_value,user_specified_id,object_number_id
,deg,deg,mas,mas,deg,,,mas,,
object,float64,float64,float32,float32,int16,str1,object,float64,object,int64
* alf Cen A,219.90205833170774,-60.83399268831004,34.17567,28.45653,90,O,2007A&A...474..653V,742.12,Alpha Cen A,1
* alf Cen B,219.89609628987276,-60.83752756558407,180.2203,171.3926,90,O,2007A&A...474..653V,742.12,Alpha Cen B,2
NAME Proxima Centauri,217.42894222160578,-62.67949018907555,0.024,0.0344,90,O,2020yCat.1350....0G,768.0665,Proxima Cen,3


In [30]:
import numpy as np
#α centauri A


ra=219.90205833170774
dec=-60.83399268831004
plx_value=742.12


γ=np.pi/180

def result_crtsn(m,n,r):
    plxarc=r/1000
    d=(1/plxarc)*3.086e16  #in m

    x=d*(np.cos(γ*n))*np.cos(γ*m)
    y=d*(np.cos(γ*n))*np.sin(γ*m)
    z=d*np.sin(γ*m)

    return (x,y,z)





In [31]:
result_crtsn(ra,dec,plx_value)

(np.float64(-1.5546447523881598e+16),
 np.float64(-1.2999796772302504e+16),
 np.float64(-2.667491257980127e+16))